In [1]:
import os
import pandas as pd
import numpy as np
import json
import pickle
from collections import defaultdict
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

import torch

import os, sys
parentPath = os.path.abspath("..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)
    
from src.data import load_source, slice_d
from src.config import Config, get_option_fallback
from knowledge_tracing.trainer import Trainer

/root/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/root/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
projectdir = Path('/code')
assert projectdir.exists()

In [8]:
results = [
    'output/1220_gt_all_assistment2009/dkt/report/20191220-2209/ksdkt.json',
    'output/1220_gt_all_assistment2009/dkt_w1w2/report/20191220-2224/ksdkt.json',
    'output/1220_gt_all_assistment2009/dkt_ks05/report/20191220-2238/ksdkt.json',
    'output/1220_gt_all_assistment2009/dkt_pre5/report/20191222-1737/ksdkt.json',
    'output/1220_gt_all_assistment2009/dkt_pre10/report/20191220-2252/ksdkt.json',
    'output/1220_gt_all_assistment2009/dkt_pre200/report/20191222-1752/ksdkt.json',
    
    'output/1220_gt_all_assistment2009/ef15/report/20191224-1437/geddkt.json',
    
    'output/1220_gt_all_assistment2009/ef15/report/20191221-0010/geddkt.json',
    'output/1220_gt_all_assistment2009/ef15_w1w2/report/20191221-0026/geddkt.json',
    'output/1220_gt_all_assistment2009/ef15_ks05/report/20191221-0042/geddkt.json',
    'output/1220_gt_all_assistment2009/ef15_pre10/report/20191221-0357/geddkt.json',
    
    'output/1220_gt_all_assistment2009/ef15_generative/report/20191221-1211/geddkt.json',
]

In [11]:
df = pd.DataFrame(columns=['dataset', 'model', 'lw1', 'lw2', 'ksv', 'pre', 'auc', 'w1', 'w2', 'ksv_score'])
df = pd.DataFrame(columns=['dataset', 'model', 'lw1', 'lw2', 'ksv', 'pre', 'bestepoch', 'auc', 'w1', 'w2', 'ksv_score'])

for res_path in results:
    with open(projectdir / res_path, 'r') as f:
        res = json.load(f)
    best_idx = np.argmax(res['indicator']['eval_auc'])
    row = {
        'dataset':   res['config']['source_data'],
        'model':     res['config']['model_name']+'/'+res['config']['exp_name'],
        'lw1':       res['config']['lambda_l1'],
        'lw2':       res['config']['lambda_l2'],
        'ksv':       res['config']['ksvector_l1'],
        'pre':       res['config']['pre_dummy_epoch_size'],
        'bestepoch': best_idx * 10,
        'auc':       res['indicator']['eval_auc'][best_idx],
        'w1':        res['indicator']['waviness_l1'][best_idx],
        'w2':        res['indicator']['waviness_l2'][best_idx],
        'ksv_score': res['indicator']['ksvector_l1'][best_idx],
    }
    df = df.append(row, ignore_index=True)
    
df

,dataset,model,lw1,lw2,ksv,pre,bestepoch,auc,w1,w2,ksv_score
0,original_ASSISTmentsSkillBuilder0910,ksdkt/dkt,0.000,0.0,0.0,0,790,0.848056,0.031516,0.003574,0.010638
1,original_ASSISTmentsSkillBuilder0910,ksdkt/dkt_w1w2,0.003,3.0,0.0,0,630,0.845646,0.016622,0.000966,0.010267
2,original_ASSISTmentsSkillBuilder0910,ksdkt/dkt_ks05,0.000,0.0,0.5,0,780,0.848250,0.030459,0.003318,0.009816
3,original_ASSISTmentsSkillBuilder0910,ksdkt/dkt_pre5,0.000,0.0,0.0,5,720,0.849502,0.030995,0.003442,0.010700
4,original_ASSISTmentsSkillBuilder0910,ksdkt/dkt_pre10,0.000,0.0,0.0,10,980,0.849444,0.034220,0.003921,0.010797
5,original_ASSISTmentsSkillBuilder0910,ksdkt/dkt_pre200,0.000,0.0,0.0,200,970,0.851842,0.045217,0.007033,0.011017
6,original_ASSISTmentsSkillBuilder0910,geddkt/ef15,0.000,0.0,0.0,0,940,0.848242,0.031478,0.003627,0.008145
7,original_ASSISTmentsSkillBuilder0910,geddkt/ef15,0.000,0.0,0.0,0,970,0.846113,0.025250,0.002502,0.008088
8,original_ASSISTmentsSkillBuilder0910,geddkt/ef15_w1w2,0.003,3.0,0.0,0,970,0.842859,0.013632,0.000629,0.007924
9,original_ASSISTmentsSkillBuilder0910,geddkt/ef15_ks05,0.000,0.0,0.5,0,990,0.846388,0.025255,0.002496,0.007829


In [12]:
print(df.to_latex())

\begin{tabular}{lllrrrllrrrr}
\toprule
{} &                               dataset &                   model &    lw1 &  lw2 &  ksv &  pre & bestepoch &       auc &        w1 &        w2 &  ksv\_score \\
\midrule
0  &  original\_ASSISTmentsSkillBuilder0910 &               ksdkt/dkt &  0.000 &  0.0 &  0.0 &    0 &       790 &  0.848056 &  0.031516 &  0.003574 &   0.010638 \\
1  &  original\_ASSISTmentsSkillBuilder0910 &          ksdkt/dkt\_w1w2 &  0.003 &  3.0 &  0.0 &    0 &       630 &  0.845646 &  0.016622 &  0.000966 &   0.010267 \\
2  &  original\_ASSISTmentsSkillBuilder0910 &          ksdkt/dkt\_ks05 &  0.000 &  0.0 &  0.5 &    0 &       780 &  0.848250 &  0.030459 &  0.003318 &   0.009816 \\
3  &  original\_ASSISTmentsSkillBuilder0910 &          ksdkt/dkt\_pre5 &  0.000 &  0.0 &  0.0 &    5 &       720 &  0.849502 &  0.030995 &  0.003442 &   0.010700 \\
4  &  original\_ASSISTmentsSkillBuilder0910 &         ksdkt/dkt\_pre10 &  0.000 &  0.0 &  0.0 &   10 &       980 &  0.849444 &  0

## 1221_gt_searchPreX_assistment2009

In [35]:
resultpath2 = sorted(projectdir.glob('output/1221_gt_searchPreX_assistment2009/*/report/*/*.json'))

In [37]:
df = pd.DataFrame(columns=['dataset', 'model', 'lw1', 'lw2', 'ksv', 'pre', 'bestepoch', 'auc', 'w1', 'w2', 'ksv_score'])

for res_path in resultpath2:
    with open(projectdir / res_path, 'r') as f:
        res = json.load(f)
    best_idx = np.argmax(res['indicator']['eval_auc'])
    row = {
        'dataset':   res['config']['source_data'],
        'model':     res['config']['model_name'],
        'lw1':       res['config']['lambda_l1'],
        'lw2':       res['config']['lambda_l2'],
        'ksv':       res['config']['ksvector_l1'],
        'pre':       res['config']['pre_dummy_epoch_size'],
        'bestepoch': best_idx * 10,
        'auc':       res['indicator']['eval_auc'][best_idx],
        'w1':        res['indicator']['waviness_l1'][best_idx],
        'w2':        res['indicator']['waviness_l2'][best_idx],
        'ksv_score': res['indicator']['ksvector_l1'][best_idx],
    }
    df = df.append(row, ignore_index=True)
    
df

,dataset,model,lw1,lw2,ksv,pre,bestepoch,auc,w1,w2,ksv_score
0,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,0,790,0.848056,0.031516,0.003574,0.010638
1,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,10,1190,0.850610,0.036189,0.004302,0.010970
2,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,100,1130,0.852440,0.044055,0.006467,0.011094
3,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,150,990,0.851169,0.045570,0.007067,0.011021
4,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,200,1080,0.854232,0.045542,0.007027,0.011062
5,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,15,1090,0.850954,0.035666,0.004176,0.010684
6,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,20,600,0.849863,0.033929,0.003678,0.010769
7,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,30,890,0.851544,0.039166,0.004868,0.010808
8,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,40,1150,0.853078,0.042221,0.005736,0.010904
9,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,5,700,0.849700,0.030599,0.003359,0.010709


In [38]:
print(df.to_latex())

\begin{tabular}{lllrrrllrrrr}
\toprule
{} &                               dataset &  model &  lw1 &  lw2 &  ksv &  pre & bestepoch &       auc &        w1 &        w2 &  ksv\_score \\
\midrule
0  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &    0 &       790 &  0.848056 &  0.031516 &  0.003574 &   0.010638 \\
1  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &   10 &      1190 &  0.850610 &  0.036189 &  0.004302 &   0.010970 \\
2  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &  100 &      1130 &  0.852440 &  0.044055 &  0.006467 &   0.011094 \\
3  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &  150 &       990 &  0.851169 &  0.045570 &  0.007067 &   0.011021 \\
4  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &  200 &      1080 &  0.854232 &  0.045542 &  0.007027 &   0.011062 \\
5  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &   15 &